### Prepare for takeoff

In [6]:
import polars as pl
import polars.selectors as cs
from helper import pivot_txs, make_transitions

In [7]:
tx_inputs = pl.read_parquet('./dfs/transaction_inputs.parquet')
tx_outputs = pl.read_parquet('./dfs/transaction_outputs.parquet')

### Q1 - How much Bitcoin was pegged-in to this federation?

**Answer**: 8.42375871 BTC

**Rationale**:  A *peg-in* is synonymous with a deposit.  A user deposits BTC to the federation's wallet and the federation mints and then sends the user an equal amount of eCash.

Since the transactions are framed from the point of view of the federation, the input side of the transaction involves a user sending BTC to the federation's wallet.  The output side involves the federation minting eCash and sending it to the user.

The only reason BTC would be sent to the federation's wallet is for the sole purpose of minting to eCash.

Thus, we only need to sum the amount of msats from the *input* transactions where `kind = 'wallet`'.

**Code to reproduce**:

In [8]:
pegged_in_btc = tx_inputs.filter(pl.col('kind') == 'wallet').sum()['amount_msat']/1e3/1e8
print(f'There was {pegged_in_btc[0]:.8f} BTC pegged-in.')

There was 8.42375871 BTC pegged-in.


### Q2 - How much Bitcoin was pegged-out from this federation?

**Answer**: 7.81928988 BTC

**Rationale**:  A *peg-out* is synonymous with a withdrawal.  A user sends the federation eCash and the federation gives the user an equal amount of BTC.

Keeping in mind the federation point of view, the input side of the transaction involves the federation receiving eCash and the output side involves the federation sending BTC from its wallet.

The only reason BTC would be sent from the federation's wallet is for the sole purpose of returning it to the user.

Thus, we only need to sum the amount of msats from the *output* transactions where `kind = 'wallet`'.

**Code to reproduce**:

In [9]:
pegged_out_btc = tx_outputs.filter(pl.col('kind') == 'wallet').sum()['amount_msat']/1e3/1e8
print(f'There was {pegged_out_btc[0]:.8f} BTC pegged-out.')

There was 7.81928988 BTC pegged-out.


### Q3 - What is the current on-chain balance?

**Answer**: 60,446,883,000 msats.

**Rationale**: Per the [get_federation_assets() function](https://github.com/douglaz/fedimint-observer/blob/c76c27abede4f26aa9414570107c3f92c2f4013a/src/federation/observer.rs#L529) in the `fediminit-observer` repo, a federation's assets is defined as:
 $$\sum_{i \, \in \, {input\_txs}}{amount\_msat_{i}} \; - \sum_{o \, \in \, {output\_txs}}{amount\_msat_{o}}$$ 
 where $kind_i = kind_o = \text{'wallet'}$.

 **Code to reproduce**:

In [10]:
in_msats = (
    tx_inputs
    .filter(
        pl.col('kind') == 'wallet',
    )
    .select('amount_msat')
    .sum()
)

out_msats = (
    tx_outputs
    .filter(
        pl.col('kind') == 'wallet',
    )
    .select('amount_msat')
    .sum()
)

fed_assets_msats = in_msats - out_msats

print(f'The on-chain balance is: {fed_assets_msats['amount_msat'][0]:,}          msats')
print(f'The on-chain balance is: {int(fed_assets_msats['amount_msat'][0]/1e3):>14,}          sats')
print(f'The on-chain balance is: {fed_assets_msats['amount_msat'][0]/1e3/1e8:>23.8f} btc')

The on-chain balance is: 60,446,883,000          msats
The on-chain balance is:     60,446,883          sats
The on-chain balance is:              0.60446883 btc
